# Refactoring

## Definition
* In essence, refactoring code is the process of making small changes to the structure of the code while maintaining its public behavior.
* The main purpose of refactoring is to increase the efficiency in the various parts of the project.
    * Programmer's efficiency - Reducing code complexity, improving readability.
    * System efficiency - Optimizing performance of the code, reducing the potential for bugs.
* For instance, take the sample Python code snippet below:

In [3]:
# Simple operation - Writing a piece of information to a log file
import os # Cleaning up created test file

# Setup sample data
logs = ["log 1: foo", "log 2: bar", "log 3: baz", "log 4: qux"]

# Appending the data
appended_logs: str = ""
for log in logs:
    appended_logs += f"{log}\n"
    
# Writing the data
opened_file = open("testlog.txt", "w")
opened_file.write(appended_logs)
opened_file.close()

# Validating the data has been written into a file
reopened_file = open(f"testlog.txt", "r")
for line in reopened_file.readlines():
    print(line)
reopened_file.close()

# Performing cleanup
os.remove("testlog.txt")

log 1: foo

log 2: bar

log 3: baz

log 4: qux



* The example above appears to be working as expected with the correct output being created from the read file - This works as a MVP.
* However, there are several opportunities for refactoring:
    * The programmer has to remember to call the close() method of the file object each time after the file is written to/ read from.
        * This creates an additional mental burden on the programmer.
        * Failure to close files may lead to dangling file handles, eventually resulting in OSErrors and/or possibility of corrupted files.
        * Purpose of refactoring: **Reducing the potential for bugs.**
    * Current method of creating strings is relatively inefficient and may cause future scalability issues.
        * Due to the immutable nature of strings in Python, appending to an existing string requires allocation of a new string instance, with the existing string being copied over in memory.
        * With a small number of log messages there is virtually no measurable impact. However, as the number of log messages increases, this will become a growing problem.
        * Purpose of refactoring: **Increasing performance.**  

In [4]:
# Refactored version

import os # Cleaning up created test file
import io

# Setup sample data
logs = ["log 1: foo", "log 2: bar", "log 3: baz", "log 4: qux"]

# Appending the data
appended_logs_buffer = io.StringIO() # Writing to a mutable buffer.
for log in logs:
    appended_logs_buffer.write(f"{log}\n") # Buffer will reallocate memory instead of performing copy
appended_logs = appended_logs_buffer.getvalue()

# Writing the data
# Using context managers will obviate the need to call the close() method.
# This make the code less error prone.
with open("testlog.txt", "w") as opened_file:
    opened_file.write(appended_logs)
    
# Validating the data has been written into a file
with open("testlog.txt", "r") as reopened_file:
    for line in reopened_file:
        print(line)

# Performing cleanup
os.remove("testlog.txt")

log 1: foo

log 2: bar

log 3: baz

log 4: qux



* To summarize, refactoring the code by re-writing certain portions of it has made it more robust and performant.
* However, in reality performance optimization are usually only be carried out after the hot code paths are identified.
    * The example above is lacking in this regard since it does performance optimization without any benchmarks.
    * Perhaps the use case is only for tens of log messages, in which case using a mutable buffer over the naive method does not yield any noticeable performance increases since other overheads may dominate.
* One import aspect to also consider during refactoring is that shorter code blocks may not always be the better option.
    * Concise code is usually preferred, but sometimes longer code blocks that lays out the explicit steps may be preferable to shorter blocks that abstract away the important details.